In [1]:
import import_ipynb
import clean_data
import pandas as pd
import modeling_2 as model
import numpy as np

In [2]:
dfs = clean_data.getDataFramesFromFiles('../CollegeScorecard_Raw_Data/')
clean_data.addYearAsLabel(dfs)
clean_data.dropUselessColumn(dfs)
clean_data.convertUnknownsToNans(dfs)
clean_data.dropColsAllNans(dfs)
merged_df = clean_data.intersection_and_merge(dfs)
merged_df = clean_data.convertMixedDataTypes(merged_df)
merged_df = merged_df.fillna(merged_df.mean())

/Users/williamegan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/williamegan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1806,1814) have mixed types. Specify dtype option on import or set low_memory=False.
  if (await self.run_code(code, result,  async_=asy)):
/Users/williamegan/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3248: DtypeWarning: Columns (1,6,442,451,598,607,611,620,624,633,789,802,806,815,971,984,997,1153,1166,1407,1408,1411,1424,1425,1426,1431,1432,1433,1437,1438,1439,1440,1445,1446,1447,1451,1452,1453,1454,1459,1460,1461,1465,1466,1467,1468,1473,1474,1475,1479,1480,1481,1482,1487,1488,1489,1501,1502,1537,1538,1539,1540,1541,1542,1606,1609,1610,1611,1613,1614,1615,1616,1688,1689,1690,1691,1692,1908,19

In [3]:
train, test = model.sampling_data(merged_df)

In [8]:
target = 'MD_EARN_WNE_P6'
est = 14
x_train, y_train, x_test, y_test = model.split_data(train, test, target)
x_train = x_train.drop(['STABBR'], axis=1)
x_test = x_test.drop(['STABBR'], axis=1)
x_train = x_train.drop(['INSTNM'], axis=1)
x_test = x_test.drop(['INSTNM'], axis=1)

In [9]:
rf_feature_importance, rf, preds = model.create_random_forest(x_train, y_train, x_test, est)

/Users/williamegan/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:737: UserWarning: Some inputs do not have OOB scores. This probably means too few trees were used to compute any reliable oob estimates.
  warn("Some inputs do not have OOB scores. "


In [11]:
rf

RandomForestRegressor(bootstrap=True, criterion='mse', max_depth=None,
                      max_features='auto', max_leaf_nodes=None,
                      min_impurity_decrease=0.0, min_impurity_split=None,
                      min_samples_leaf=1, min_samples_split=2,
                      min_weight_fraction_leaf=0.0, n_estimators=14,
                      n_jobs=None, oob_score=True, random_state=None, verbose=0,
                      warm_start=False)

Proposed engineering:
- make AANAPII (asian and pacific islander flag) and other flags for specific things all into one categorical variable


- I wonder what to do about all the different ACT scores. I think just pick like the midpoint cumulative(ACTCMMID)
    - alternatively, could separate by subject as well, since field of study is something interesting to us 
    - ACTENMID, ACTMTMID, ACTWRMID


- remove ADM_RATE_ALL? superfluous ficen ADM_RATE


-  I kind of want to combine C150_4_POOLED_SUPP, C150_L4_POOLED_SUPP into one feature so that it is more populated. if there is a 4 or two year indicator maybe we can do that (but some schools do both!)


- COSTT4_A and COSTT4_P are only different in the academic year vs program year thing. I wonder if it wo uld behoove us to have a binary variable for one or the other type of program, and then combine this data into one category, or if it is better to just leave them apart


- D150_4_AIANOLD D150_4_HISPOLD, D150_L4_AIANOLD D150_L4_HISPOLD
    - these boys also are divided by 4 and L4. I am starting to think that one feature for a tag of L4 and 4 would be nice, and then all the features that depend on that can be combined, and the flag will take care of the distinction for all of them :)


- There is a bunch of information about dependence and their family income level. I cannot think of a way to combine these, and, in fact, we may not want to since we have already seen that these can be powerful predictive features.



- maybe faminc can be combined? i dont think so and i dont think we want to anyway


- GRADS is share of female grads. maybe rename the variable


- keep debt for various incomes intact

- INC_PCT_H1 and variants is for aided students an d thier family incomes
    - also the independent version of these is IND_INC_PCT_H1
    
    
- for all the NPT features, we have the same income brackets, but different kinds of institutions. (average net price is the information)
    - i suggest getting some tags for what kind of school it is, and then commpress these into 4 columns ajust based on the different income brackets. Should be reductive 
    
    
- OPEFLAG is titel IV eligibility


- PAR_ED_PCT_HS. Percentage of parents that got various kinds of education. cant think of a way to compress these atm, they all are interesting and every school should have a value there


- for these PCIP boys, we can try to bucket them by subject. Not sure if this is good or bad for prediction, but it sure will reduce the number of columns


- preddeg?


- SATMTMID, SATVRMID, SATWRMID. same issue as with act. I am thinking just the midpoints, no percentiles
    - dont forget SAT_AVG vs SAT_AVG_ALL


- UG is all share of undergraduates of certain demographics. key!
    - difference between UG and UGDS is just the 'degree-seeking' component
    -  we may be able to get rid of one or the other type. If we were to keep one it should be UGDS
    









In [9]:
for i in merged_df.columns:
    print(i)

AANAPII
ACTCM25
ACTCM75
ACTCMMID
ACTEN25
ACTEN75
ACTENMID
ACTMT25
ACTMT75
ACTMTMID
ACTWR25
ACTWR75
ACTWRMID
ADM_RATE
ADM_RATE_ALL
AGEGE24
AGE_ENTRY
ANNHI
AVGFACSAL
C150_4_POOLED_SUPP
C150_L4_POOLED_SUPP
CCBASIC
CCSIZSET
CCUGPROF
CONTROL
COSTT4_A
COSTT4_P
CURROPER
D150_4_AIANOLD
D150_4_HISPOLD
D150_L4_AIANOLD
D150_L4_HISPOLD
DEPENDENT
DEP_INC_AVG
DEP_INC_PCT_H1
DEP_INC_PCT_H2
DEP_INC_PCT_LO
DEP_INC_PCT_M1
DEP_INC_PCT_M2
DEP_STAT_PCT_IND
DISTANCEONLY
D_PCTPELL_PCTFLOAN
FAMINC
FAMINC_IND
FEMALE
FIRST_GEN
GRADS
HBCU
HIGHDEG
HI_INC_DEBT_MDN
HSI
ICLEVEL
INC_PCT_H1
INC_PCT_H2
INC_PCT_LO
INC_PCT_M1
INC_PCT_M2
IND_INC_AVG
IND_INC_PCT_H1
IND_INC_PCT_H2
IND_INC_PCT_LO
IND_INC_PCT_M1
IND_INC_PCT_M2
INEXPFTE
INSTNM
LO_INC_DEBT_MDN
MD_EARN_WNE_P6
MD_FAMINC
MD_INC_DEBT_MDN
MEDIAN_HH_INC
MENONLY
NANTI
NPT41_OTHER
NPT41_PRIV
NPT41_PROG
NPT41_PUB
NPT42_OTHER
NPT42_PRIV
NPT42_PROG
NPT42_PUB
NPT43_OTHER
NPT43_PRIV
NPT43_PROG
NPT43_PUB
NPT44_OTHER
NPT44_PRIV
NPT44_PROG
NPT44_PUB
NPT45_OTHER
NPT45_PRIV
NPT4